# Movies dataset

## Contexto

Esta base de dados consiste de filmes lançados até julho de 2017. Os dados incluem elenco, equipe, gênero, orçamento, receita, datas de lançamento, idiomas, empresas de produção, países, contagem de votos e média de votos.

## Referências

- https://www.kaggle.com/rounakbanik/the-movies-dataset
- https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system

## Análise exploratória dos dados

O dataset que iremos trabalhar possui os seguintes (principais) atributos:

* **budget** - The budget in which the movie was made.
* **genre** - The genre of the movie, Action, Comedy ,Thriller etc.
* **homepage** - A link to the homepage of the movie.
* **id** - This is infact the movie_id as in the first dataset.
* **keywords** - The keywords or tags related to the movie.
* **original_language** - The language in which the movie was made.
* **original_title** - The title of the movie before translation or adaptation.
* **overview** - A brief description of the movie.
* **popularity** - A numeric quantity specifying the movie popularity.
* **production_companies** - The production house of the movie.
* **production_countries** - The country in which it was produced.
* **release_date** - The date on which it was released.
* **revenue** - The worldwide revenue generated by the movie.
* **runtime** - The running time of the movie in minutes.
* **status** - "Released" or "Rumored".
* **tagline** - Movie's tagline.
* **title** - Title of the movie.
* **vote_average** - average ratings the movie recieved.
* **vote_count** - the count of votes recieved.

In [ ]:
# importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Leitura e limpeza dos dados

In [ ]:
# leitura dos dados
df = pd.read_csv("https://pycourse.s3.amazonaws.com/movies.csv")
df.head()

In [ ]:
# descritivo dos dados
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title='Movies dataset')

In [ ]:
profile

In [ ]:
# info
df.info()

In [ ]:
# limpeza: remoção de colunas com poucas entradas válidas
df.drop(['belongs_to_collection',
         'homepage',
         'tagline'],
         axis='columns',
         inplace=True)

In [ ]:
# limpeza: remoção de colunas com pouca variabilidade ou
# irrelevante para a análise
df.drop(['adult', 'overview'],
        axis='columns',
        inplace=True)

In [ ]:
# info
df.info()

In [ ]:
# limpeza: remoção de linhas com dados faltantes
df.dropna(axis='index', inplace=True)

In [ ]:
# info
df.info()

In [ ]:
# dados filtrados
df.head()

## Estruturando os dados

Alguns atributos apresentam a seguinte estrutura: `[{'id': id, 'name': name}]`.

Precisamos definir uma estrutura mais simples para análise...

In [ ]:
# estrutura original
df[['genres',
    'production_countries',
    'spoken_languages']].head()

Iremos codificar esses atributos através da operação de one-hot-encoding:

![ohe-hot-enconding](https://pycourse.s3.amazonaws.com/ohe.png)

In [ ]:
import json

def list_to_ohe(df: pd.DataFrame, cols: list):

  # para cada coluna
  n_rows = df.shape[0]
  df.reset_index(inplace=True, drop=True)
  for col_i in cols:

    # selecionado a coluna
    dfi = df[col_i]

    # dicionário para mapeamento
    new_cols = {}

    # percorrendo cada linha do dataframe
    for i, row in enumerate(dfi):

      # leitura da string como um JSON
      row = row.replace("\'", "\"")
      list_i = json.loads(row)

      # percorrendo cada elemento da lista
      for elem in list_i:
      
        # nova coluna com a categoria
        new_col_name = col_i + '_' + elem['name']  # col_[nome_categoria]

   
        if new_col_name not in new_cols:
          new_cols[new_col_name] = np.zeros((n_rows))
        
        # atribui classificação
        new_cols[new_col_name][i] = 1
    
    # append new columns     # adiciona nova coluna
        if new_col_name not in new_cols:
          new_cols[new_col_name] = np.zeros((n_rows))
        
        # atribui classificação
        new_cols[new_col_name][i] = 1
    
    # append new columns
    new_df = pd.DataFrame(new_cols)
    df = pd.concat([df, new_df],
                   axis=1).reset_index(drop=True) 
  
  return df

In [ ]:
# colunas para transformar
cols_to_transform = ['genres']

print('Shape antes:', df.shape)
df = list_to_ohe(df, cols=cols_to_transform)
print('Shape depois:', df.shape)

In [ ]:
# verificando novas colunas de generos de filmes
genres_attr = [col for col in df if col.startswith('genres_')]
print("Colunas de gênero de filme inseridas:\n",
      np.array(genres_attr).reshape(-1, 1))

In [ ]:
# dataframe tratado
df.head()

## Análise

### Definindo um score

Precisamos definir um score para comparar avaliações entre filmes, para levar em conta a quantidade de avaliações que cada filme recebeu. Para isso, será utilizada a fórmula do IMDB para definição do score de um filme:

$$\mathrm{Weighted~Rating~(WR)} = \frac{v}{v+m}R + \frac{m}{v+m}C,$$

onde:

- $v$ é o número de avaliações (**vote_count**);
- $m$ é o número mínimo de avaliações necessárias para contabilização;
- $R$ é a nota média do filme (**vote_average**);
- $C$ é a média de todas as notas.

A variável $C$ pode ser calculada da seguinte maneira:

In [ ]:
# quantidade de filmes sem avaliação
cond = df['vote_count'] < 1e-3 # 0.001
print("Quantidade de filmes sem avaliação:", sum(cond))

In [ ]:
# removendo da análise filmes que não receberam avaliações
df = df.loc[~cond]
print("Novo shape:", df.shape)

In [ ]:
# verificando operação
cond = df['vote_count'] < 1e-3
print("Quantidade de filmes sem avaliação:", sum(cond))

In [ ]:
# C: nota média entre todos os filmes
C = df['vote_average'].mean()
print("Média de todas as notas (C):", C)

O número mínimo de votos pode ser obtido a partir dos percentis de **vote_count**. Iremos levar em consideração na análise somente os filmes que receberam mais votos que pelo menos 75\%  dos filmes da lista.

In [ ]:
# estatísticas dos votos
df[['vote_count']].describe()

In [ ]:
# m: número mínimo de votos para análise
m = df['vote_count'].quantile(0.9)
print("Número mínimo de votos (m):", m)

In [ ]:
# filtrando o dataset
df = df[df['vote_count'] > m]

Cálculo do score:

In [ ]:
# adição da nova coluna com o score
v = df['vote_count']
R = df['vote_average']
df.loc[:, 'score'] = v/(v+m) * R + m/(m+v) * C

In [ ]:
# ordenando o dataframe pelo score calculado
df.sort_values(by='score',
               ascending=False,
               inplace=True)
df.reset_index(inplace=True)

### TOP 10: score

In [ ]:
# TOP 10: score
df[['original_title', 'vote_count', 'vote_average', 'score']].head(10)

In [ ]:
# gênero de filmes no TOP 10 de score
df_top_score = df[:10]

# contagem de gêneros
df_top_score_gen = df_top_score[genres_attr].sum().sort_values(ascending=False)
df_top_score_gen

In [ ]:
# color map
from matplotlib import cm
cmap = cm.get_cmap('Set3')

In [ ]:
# retirando valores nulos
df_top_score_gen = df_top_score_gen[df_top_score_gen > 0]

# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_score_gen.index]

# plot
df_top_score_gen.plot.pie(autopct='%1.1f%%',
                          pctdistance=0.8,
                          radius=1.25,
                          labels=labels,
                          cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (score)');

### TOP 10: popularidade

In [ ]:
# ordenando o dataframe pela popularidade
df['popularity'] = df['popularity'].astype(float) 
df_top_pop = df.sort_values(by='popularity',
                            ascending=False)

In [ ]:
# TOP 10: popularidade
df_top_pop[['original_title', 'score', 'popularity']].head(10)

In [ ]:
# visualização dos filmes mais populares
df_top_pop[:10].plot.barh(x='original_title',
                          y=['popularity'])
plt.gca().invert_yaxis();

In [ ]:
# gênero de filmes no TOP 10 de popularity
df_top_pop_gen = df_top_pop[genres_attr]
df_top_pop_gen = df_top_pop_gen[:10].sum().sort_values(ascending=False)

# filtro em valores maiores que 0
df_top_pop_gen = df_top_pop_gen[df_top_pop_gen > 0]
df_top_pop_gen

In [ ]:
# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_pop_gen.index]

# plot
df_top_pop_gen.plot.pie(autopct='%1.1f%%',
                          pctdistance=0.8,
                          radius=1.25,
                          labels=labels,
                          cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (popularidade)');

### TOP 10: receita

In [ ]:
# ordenando o dataframe pela receita
df['revenue'] = df['revenue']/1e6  # em milhões de dólares
df_top_rev = df.sort_values(by='revenue',
                            ascending=False)

In [ ]:
# TOP 10: receita
df_top_rev[['original_title', 'score', 'popularity', 'revenue']].head(10)

In [ ]:
# gênero de filmes no TOP 10 de revenue
df_top_rev_gen = df_top_rev[genres_attr]
df_top_rev_gen = df_top_rev_gen[:10].sum().sort_values(ascending=False)

# filtro em valores maiores que 0
df_top_rev_gen = df_top_rev_gen[df_top_rev_gen > 0]
df_top_rev_gen

In [ ]:
# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_rev_gen.index]

# plot
df_top_rev_gen.plot.pie(autopct='%1.1f%%',
                        pctdistance=0.8,
                        radius=1.25,
                        labels=labels,
                        cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (receita)');